In [1]:
from glob import glob
import os
import joblib
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
#Test with SET files
def read_data_test(file_path):
    data=mne.io.read_raw_eeglab(file_path,preload=True)
    data.set_eeg_reference()
    #data.filter(l_freq=0.5,h_freq=45)
    epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
    array=epochs.get_data()
    return array

In [3]:
#Test with EDF files
def read_data_test_edf(file_path):
    data=mne.io.read_raw_edf(file_path,preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=0.5,h_freq=45)
    epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
    array=epochs.get_data()
    return array

In [4]:
from scipy import stats

def mean(data):
    return np.mean(data,axis=-1)
    
def std(data):
    return np.std(data,axis=-1)

def ptp(data):
    return np.ptp(data,axis=-1)

def var(data):
        return np.var(data,axis=-1)

def minim(data):
      return np.min(data,axis=-1)


def maxim(data):
      return np.max(data,axis=-1)

def argminim(data):
      return np.argmin(data,axis=-1)


def argmaxim(data):
      return np.argmax(data,axis=-1)

def mean_square(data):
      return np.mean(data**2,axis=-1)

def rms(data): #root mean square
      return  np.sqrt(np.mean(data**2,axis=-1))  

def abs_diffs_signal(data):
    return np.sum(np.abs(np.diff(data,axis=-1)),axis=-1)

def skewness(data):
    return stats.skew(data,axis=-1)

def kurtosis(data):
    return stats.kurtosis(data,axis=-1)

def concatenate_features(data):
    return np.concatenate((mean(data),std(data),ptp(data),var(data),minim(data),maxim(data),argminim(data),argmaxim(data),
                          mean_square(data),rms(data),abs_diffs_signal(data),
                          skewness(data),kurtosis(data)),axis=-1)

In [5]:
test_dict={0:'Sick',1:'Healthy'}

In [6]:
test_file_path=glob('derivatives/F/*.set')
print(len(test_file_path))

23


In [7]:
%%capture
sample_data_pred=read_data_test(test_file_path[3])

In [8]:
from tqdm import tqdm_notebook
featuress=[]
for data in tqdm_notebook(sample_data_pred):
    featuress.append(concatenate_features(data))
featuress=np.array(featuress)
featuress.shape

C:\Users\91904\AppData\Local\Temp\ipykernel_14396\2936894702.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for data in tqdm_notebook(sample_data_pred):


  0%|          | 0/158 [00:00<?, ?it/s]

(158, 247)

In [9]:
loaded_model = joblib.load('models/LR_SC.sav')
test_dict.get(np.bincount(loaded_model.predict(featuress)).argmax())

'Sick'

In [10]:
loaded_model = joblib.load('models/LDA_Kfold_SC.sav')
test_dict.get(np.bincount(loaded_model.predict(featuress)).argmax())

'Sick'

In [11]:
loaded_model = joblib.load('models/XGboost_SC.sav')
test_dict.get(np.bincount(loaded_model.predict(featuress)).argmax())

'Sick'